# Challenge Problem: Mask Detector

## 0- Using Rekognition to do person detection (Editing required)

In [ ]:
import boto3
rekog_client = boto3.client('rekognition')

## Try to use AWS Rekognition API to detect if a person is in the image
## Refer to https://docs.aws.amazon.com/rekognition/latest/dg/labels-detect-labels-image.html
## Then use Cuomtom Lables endpoint to detect if there is mask in the image 
## Combine the results of two predictions we can build auto mask detector

def is_there_a_person_in_the_image(bucket,photo)
    ## Add your code here
    
    return True

## 1- Start your Rekognition model (Editing required)

In [ ]:
!aws rekognition start-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:230950097573:project/ppe/version/ppe.2020-07-10T08.50.41/1594389041609" \
  --min-inference-units 1 \
  --region us-east-1

## 2.1 - Define object detection inference function (**NO** Editing required)

In [ ]:
confidence =15
import boto3
import io
%matplotlib inline
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

def show_custom_labels_detector(model,bucket,photo):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=1,
        ProjectVersionArn=model)
   
   
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
    
    print(response)

    colors = ('#FF0000','#0000FF','#FF00FF','#F1C40F')
    dict1 = {'earmuffs':0,'gloves':1,'goggle':2,'mask':3}  
    # calculate and display bounding boxes for each detected custom label       
    for customLabel in response['CustomLabels']:
        if customLabel['Confidence'] >= confidence:            
            if 'Geometry' in customLabel:
                box = customLabel['Geometry']['BoundingBox']
                left = imgWidth * box['Left']
                top = imgHeight * box['Top']
                width = imgWidth * box['Width']
                height = imgHeight * box['Height']
                draw.text((left + width-20,top-15), customLabel['Name'], fill= colors[dict1[customLabel['Name']]])
                draw.text((left,top-15), str(customLabel['Confidence'])[:6], fill= colors[dict1[customLabel['Name']]],stroke_width=3)#, font='Arial.ttf') 

                points = (
                    (left,top),
                    (left + width, top),
                    (left + width, top + height),
                    (left , top + height),
                    (left, top))
                draw.line(points, fill= colors[dict1[customLabel['Name']]], width=5)

    image.show()
    display(image)
    image = image.convert('RGB')
    image.save('inference.jpg',optimize=True,quality=300)

    return response['CustomLabels']

## 2.2 - Define classification inference function (**NO** Editing required)

In [ ]:
def show_custom_labels_classifier(model,bucket,photo):
     

    client=boto3.client('rekognition')

    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)
    
    #Call DetectCustomLabels 
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=1,
        ProjectVersionArn=model)
   
   
    imgWidth, imgHeight = image.size  
    draw = ImageDraw.Draw(image)  
    
    #print(response)

    colors = ('#FF0000','#0000FF','#FF00FF','#F1C40F')
    label = response['CustomLabels'][0]
    
    draw.text((20,15), label['Name']+' '+str(label['Confidence'])[:5], fill= '#FF0000')
    
    image.show()
    display(image)
    image = image.convert('RGB')
    image.save('inference.jpg',optimize=True,quality=300)

    return response['CustomLabels']

## 3.1 - Object Detect Tools (Editing required)

In [ ]:
bucket="custom-labels-console-us-east-1-2bee2f52c5"
model= "arn:aws:rekognition:us-east-1:230950097573:project/mask-challenge/version/mask-challenge.2020-07-20T15.02.28/1595275348912"
photo = 'mask_images/Valid/1.jpg'

# Example:

# bucket = 'bucket-name'
# model = "arn:aws:rekognition:us-east-1:745137163025:project/cop/version/cop.2020-01-31T12.17.29/1580494650055"
# photo = 'pictures/4.png'

labels = show_custom_labels_detector(model,bucket,photo)

## 3.2 - Classification Tools (Editing required)

In [ ]:
bucket="custom-labels-console-us-east-1-2bee2f52c5"
model= "arn:aws:rekognition:us-east-1:230950097573:project/mask-challenge/version/mask-challenge.2020-07-20T11.57.25/1595264245587"
photo = 'mask_images/Valid/mask-on-2/1.jpg'
photo = 'mask_images/Valid/1.jpg'

# Example:

# bucket = 'bucket-name'
# model = "arn:aws:rekognition:us-east-1:745137163025:project/cop/version/cop.2020-01-31T12.17.29/1580494650055"
# photo = 'pictures/4.png'

labels = show_custom_labels_classifier(model,bucket,photo)
print('Mask status=',labels[0]['Name'],' Confidence=',str(labels[0]['Confidence'])[:5],"%")

## 4.- Combine the results of two predictors and give final prediction results (Editing required)

In [ ]:
bucket="custom-labels-console-us-east-1-2bee2f52c5"
model= "arn:aws:rekognition:us-east-1:230950097573:project/mask-challenge/version/mask-challenge.2020-07-20T11.57.25/1595264245587"
photo_prefix = 'mask_images/Valid/'

## Define the logic of combine prediction results from person detector and mask detector
def combine_results(result_person, result_mask)
    ## Add your code here
    return True

for i in range(10):
    photo = photo_prefix + str(i) +'.jpg'
    ## Add your code here
    result_person = is_there_a_person_in_the_image(bucket,photo)
    result_mask = show_custom_labels_xxx(model,bucket,photo)
    final_result = combine_result(result_person, result_mask)
    print("Detection result for image {} is {}".format(i,final_result))

## 5- Stop your Rekognition model.

In [ ]:
# After pasting remove the following line: --min-inference-units 1 \

!aws rekognition stop-project-version \
  --project-version-arn "arn:aws:rekognition:us-east-1:230950097573:project/project2/version/project2.2020-07-20T12.19.03/1595240343949" \
  --region us-east-1